### Install Libaray

In [ ]:
!pip install llama-index requests llama-index-vector-stores-chroma
!pip install git+https://github.com/Ssojux2/clova-llama-index.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.7/808.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 50.2 MB/s eta 0:

### Llama-index 기본 구현


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import chromadb
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    SimpleKeywordTableIndex,
    TreeIndex,
    Settings
)
from llama_index.vector_stores.chroma import ChromaVectorStore
from IPython.display import Markdown, display

from clova_llama_index import ClovaClient, ClovaIndexEmbeddings, ClovaLLM
from llama_index.core.prompts.base import PromptTemplate

In [ ]:
client = ClovaClient(api_key="")

### 외부 데이터 가져오기

### Embedding 설정

In [ ]:
# 임베딩 함수 초기화 후 컬렉션에 제공
embed_model = ClovaIndexEmbeddings(client, embed_batch_size=1)

Settings.embed_model = embed_model
Settings.chunk_size = 1024


In [ ]:
# Settings에 custom LLM 및 임베딩 모델 설정
Settings.llm = ClovaLLM(client)


### 벡터DB 불러오기

In [ ]:
# load_index_from_storage로 변경

# load from disk
db1 = chromadb.PersistentClient(path="/content/drive/MyDrive/localtaxonboardingchatbot/vector_db/localtaxlawdb")
chroma_collection1 = db1.get_or_create_collection("localtaxlaw")
vector_store1 = ChromaVectorStore(chroma_collection=chroma_collection1)

index1 = VectorStoreIndex.from_vector_store(
    vector_store1,
    embed_model=embed_model,
)

"""# load from disk
db2 = chromadb.PersistentClient(path="/content/drive/MyDrive/localtaxonboardingchatbot/vector_db/manual_db")
chroma_collection2 = db2.get_or_create_collection("precedents")
vector_store2 = ChromaVectorStore(chroma_collection=chroma_collection2)

index2 = VectorStoreIndex.from_vector_store(
    vector_store2,
    embed_model=embed_model,
)"""

# load from disk
db3 = chromadb.PersistentClient(path="/content/drive/MyDrive/localtaxonboardingchatbot/vector_db/manual_db")
chroma_collection3 = db3.get_or_create_collection("manual")

vector_store3 = ChromaVectorStore(chroma_collection=chroma_collection3)

index3 = VectorStoreIndex.from_vector_store(
    vector_store3,
    embed_model=embed_model,
)

In [ ]:
# Query Data from the persisted index
query_engine1 = index1.as_query_engine()
response1 = query_engine1.query("재산세 분리과세는 어떤 식으로 되나요?")
display(Markdown(f"{response1}"))
print("Document count in localtaxlaw collection:", chroma_collection1.count())

재산세 분리과세에 대한 내용은 지방세법에서 규정하고 있으며, 분리과세 주택임대소득에 대한 내용은 다음과 같습니다.

- 분리과세 주택임대소득이 있는 거주자의 종합소득에 대한 개인지방소득세 결정세액은 다음 각 호의 세액 중 하나를 선택하여 적용합니다.
  - 「소득세법」 제14조제3항제7호를 적용하기 전의 종합소득에 대한 개인지방소득세 결정세액
  - 다음 각 목의 세액을 더한 금액
     - 분리과세 주택임대소득에 대한 사업소득금액에 1천분의 14를 곱하여 산출한 금액
     - 가목 외의 종합소득에 대한 개인지방소득세 결정세액

- 제10항제2호가목에 따른 분리과세 주택임대소득에 대한 사업소득금액은 총수입금액에서 필요경비(총수입금액의 100분의 50으로 한다)를 차감한 금액으로 하되, 분리과세 주택임대소득을 제외한 해당 과세기간의 종합소득금액이 2천만원 이하인 경우에는 추가로 200만원을 차감한 금액으로 합니다.
   - 대통령령으로 정하는 임대주택을 임대하는 경우에는 해당 임대사업에서 발생한 사업소득금액은 총수입금액에서 필요경비(총수입금액의 100분의 60으로 한다)를 차감한 금액으로 하되, 분리과세 주택임대소득을 제외한 해당 과세기간의 종합소득금액이 2천만원 이하인 경우에는 추가로 400만원을 차감한 금액으로 합니다.

분리과세 주택임대소득에 대한 종합소득 결정세액의 계산 및 임대주택 유형에 따른 사업소득금액의 산출방법 등에 필요한 사항은 대통령령으로 정합니다.

Document count in localtaxlaw collection: 1772


In [ ]:
"""# Query Data from the persisted index
query_engine2 = index2.as_query_engine()
response2 = query_engine2.query("관세법이란 무엇인가요")
display(Markdown(f"{response2}"))
print("Document count in localtax collection:", chroma_collection2.count())

관세법은 관세의 부과ᆞ징수 및 수출입물품의 통관을 적정하게 하고 관세수입을 확보함으로써 국민경제의 발전에 이바지함을 목적으로 하는 법입니다. 이 법에서는 수입, 수출, 반송 등의 용어를 정의하고 있으며, 관세 부과와 징수, 통관 등에 대한 다양한 규정을 포함하고 있습니다. 또한, 관세행정에 필요한 연구개발사업 및 전문인력 양성 등 과학기술진흥을 위한 시책도 마련하여 추진할 수 있습니다.

Document count in localtax collection: 872


In [ ]:
# Query Data from the persisted index
query_engine3 = index3.as_query_engine()
response3 = query_engine3.query("납세고지서의 발급 시기는 언제인가요? 그리고 납세고지서가 그 기간 내에 발급되지 않으면 효력이 없나요?")
display(Markdown(f"{response3}"))
print("Document count in localtax collection:", chroma_collection3.count())

납세고지서의 발급 시기는 다음 각 호의 구분에 따릅니다.
  1. 납부기한이 일정한 경우: 납기가 시작되기 5일 전
  2. 납부기한이 일정하지 아니한 경우: 부과결정을 한 때
  3. 법령에 따라 기간을 정하여 징수유예 등을 한 경우: 그 기간이 만료한 날의 다음 날

그리고 납세고지서가 그 기간 내에 발급되지 않더라도 납세의무의 성립이나 송달의 효력이 좌우된다고 볼 수 없습니다. 이는 대법원 2016. 4. 15. 선고 2016두31074 판결에서도 확인할 수 있습니다.

Document count in localtax collection: 11527


### Query Engine Tool 구성하기
* 제작한 RAG용 Engine을 Tool로 변경하기

In [ ]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.prompts import PromptTemplate
from llama_index.core.response_synthesizers import TreeSummarize

# (solution) tree_summarize에서 영문으로 번역되는 현상 방지
korean_summary_prompt = PromptTemplate(
    """다음은 여러 문단으로 이루어진 정보입니다.
이 내용을 바탕으로 간결하고 정확한 요약을 한국어로 작성해주세요.

문단 목록:
{context_str}

요약:"""
)


index1_engine = index1.as_query_engine(
    response_synthesizer=TreeSummarize(summary_template=korean_summary_prompt),
    response_mode="tree_summarize", use_async=True, name="vector"
)
"""index2_engine = index2.as_query_engine(
    response_synthesizer=TreeSummarize(summary_template=korean_summary_prompt),
    response_mode="tree_summarize", use_async=True, name="vector"
)"""
index3_engine = index3.as_query_engine(
    response_synthesizer=TreeSummarize(summary_template=korean_summary_prompt),
    response_mode="tree_summarize", use_async=True, name="vector"
)


In [ ]:
# (solution) 컬렉션 설명에 관련기관을 기술하여 라우팅 성능 향상

index1_tool = QueryEngineTool.from_defaults(
    query_engine=index1_engine,
    name="localtaxlaw",
    description=(
        "분야 : 지방세 관계법령 조문"
        "지방세기본법, 지방세징수법, 지방세법, 지방세특례제한법, 지방행정제재부과금의 징수 등에 관한 법률 등 지방세 관계법령으로, 법률, 시행령, 시행규칙 및 훈령, 예규까지를 모두 포함합니다."
    ),
)

"""index2_tool = QueryEngineTool.from_defaults(
    query_engine=index2_engine,
    name="kcs",
    description=(
        "관련기관 : 관세청"
        "이 법은 관세의 부과ㆍ징수 및 수출입물품의 통관을 적정하게 하고 관세수입을 확보함으로써 국민경제의 발전에 이바지함을 목적으로 한다."
    ),
)"""
index3_tool = QueryEngineTool.from_defaults(
    query_engine=index3_engine,
    name="manual",
    description=(
        "분야 : 지방세 실무"
        """지방세법에서 다루는 취득세, 지방소득세, 재산세, 등록면허세, 레저세, 담배소비세, 지방소비세, 주민세, 자동차세, 지역자원시설세, 지방교육세 와 같은 세목을 법령 및 유권해석, 판례 등에 근거하여 운영하는 방법과
        지방세기본법 상의 납세의무와 부과, 납세환금 및 담보, 지방세와 다른 채권의 관계, 납세자의 권리, 이의신청 및 심판청구, 처벌, 과세자료 제출 및 관리, 지방세 정보화 등에 대한 구체적 지침,
        지방세징수법 상의 납세고지, 징수, 징수유예 및 압류 등 체납처분의 구체적 절차와 사례별 적용방안,
        지방세특례제한법 상의 특례 및 감면 운영 관련하여 농어업, 사회복지, 교육 및 과학기술, 문화및 관광, 기업구조 및 재무조정, 수송 및 교통에 대한 지원, 국토 및 지역개발, 공공행정 등에 대한 세제지원으로서 지방세 감면, 특례 등의 구체적 제도 운영 설명 등을 포함합니다."""
    ),
)


### Router Engine 구성하기

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    [index1_tool, index3_tool], #    [index1_tool, index2_tool, index3_tool],
    index_cls=VectorStoreIndex,
)

### 소관부처 분류하기

In [ ]:
# (solution) 질문을 받아서 어떤 질문 분야에 속하는지 미리판단하여 라우팅 성능 향상

agency_descriptions = {
    "법령 검색": "지방세기본법, 지방세징수법, 지방세법, 지방세특례제한법, 지방행정제재부과금의 징수 등에 관한 법률 등 지방세 관계법령으로, 법률, 시행령, 시행규칙 및 훈령, 예규의 조문 내용을 질문합니다.",
    "지방세 실무": "실무 사례나 유권 해석 요청, 민원 및 업무 처리 방식에 대한 질문으로서 지방세기본법, 지방세징수법, 지방세법, 지방세특례제한법에서 다루는 지방세 실무 적용 시 법령, 판례, 유권해석을 들어 어떻게 업무를 처리하는지 질문합니다."
}

In [ ]:
def build_augmented_prompt(user_question: str, llm, agency_descriptions: dict) -> str:
    """
    LLM을 사용해 관련 기관을 판단하고,
    질문 내용과 함께 질문을 포함한 프롬프트를 생성하는 함수.

    Args:
        user_question (str): 사용자 질문
        llm: 설정된 LLM (예: Settings.llm)
        agency_descriptions (dict): {"질문 분야": "질문 분야 소개글"} 형태의 사전

    Returns:
        str: 기관 소개가 포함된 최종 프롬프트
    """
    # 1. 시스템 프롬프트 생성
    system_prompt = f'''당신은 대한민국 지방세 전문가로서, 지방세 담당 공무원의 업무 처리를 적극적으로 지원합니다.
    아래는 당신이 받게 될 질문 주제의 목록입니다:
    {chr(10).join([f"- {name}: {desc}" for name, desc in agency_descriptions.items()])}
    사용자가 아래와 같은 질문을 했습니다. 아래 3가지 정보를 추출해주세요
    1. 이 질문은 어느 질문 분야와 가장 관련이 깊은가요?
    2. 사용자의 질문 주제을 질문 분야별로 다음 중 하나로 분류해주세요:
    - 법령 검색 : 지방세기본법, 지방세징수법, 지방세법, 지방세특례제한법, 지방행정제재부과금의 징수 등에 관한 법률
    - 지방세 실무 : 취득세, 지방소득세, 재산세, 등록면허세, 레저세, 담배소비세, 지방소비세, 주민세, 자동차세, 지역자원시설세, 지방교육세, 납세의무와 부과, 납세환금 및 담보, 지방세와 다른 채권의 관계, 납세자의 권리, 이의신청 및 심판청구, 처벌, 과세자료 제출 및 관리, 지방세 정보화,납세고지, 징수, 체납처분, 특례, 감면
    3. 추출정보의 신뢰도를 0~100 사이의 숫자로 함께 제공해주세요.

    출력 형식은 다음과 같이 해주세요 (각 항목은 한 줄에):
    질문 분야 :
    주제:
    신뢰도: 0~100 사이 숫자'''

    # 2. LLM을 통해 기관 추론
    response = llm.complete(system_prompt)
    agency_name = response.text.strip()

    # 3. 소개글 가져오기
    agency_intro = agency_descriptions.get(agency_name, "관련된 정보가 없습니다.")

    # 4. 최종 프롬프트 구성
    augmented_prompt = f"[지방세 관련 질문 분야: {agency_name}]\n{agency_intro}\n\n질문: {user_question}"

    return augmented_prompt


In [ ]:
from llama_index.core.query_engine import ToolRetrieverRouterQueryEngine
from llama_index.core.chat_engine import CondenseQuestionChatEngine


# (solution) 최종답변 템플릿(템플릿을 통해 답변 최적화)
korean_condense_prompt = PromptTemplate(
    """당신은 대한민국 지방세 전문가로서, 지방세 담당 공무원의 업무 처리를 적극적으로 지원합니다.
모든 질문과 응답은 반드시 한국어로 작성되어야 합니다. 필요한 경우 질문 분야를 정확하게 활용하세요.

아래는 사용자와의 대화 내역이며, 마지막에 추가 질문이 포함되어 있습니다.
추가 질문을 맥락에 맞는 독립적인 문장으로 재작성해주세요.

대화 내역:
{chat_history}

추가 질문:
{question}

독립적인 질문:"""
)

# 1. 라우팅 기반 QueryEngine 만들기
retriever = obj_index.as_retriever()
query_engine = ToolRetrieverRouterQueryEngine(retriever=retriever)

# (solution) 연속적 대화를 위해 챗봇엔진 채택
# 2. CondenseQuestionChatEngine 구성
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=korean_condense_prompt,
    verbose=True
)


In [ ]:
user_question = "산업단지개발사업 및 산업기술단지의 사업시행자가 보유하는 부동산에 대해서 감면하는 경우도 있나요?"
augmented_prompt = build_augmented_prompt(user_question, Settings.llm, agency_descriptions)
print("[Augmented Prompt]\n")
print(augmented_prompt)
print("\n" + "="*80 + "\n")

response = chat_engine.chat(augmented_prompt)
print("\n[ChatEngine 응답]\n")
print(response)

[Augmented Prompt]

[지방세 관련 질문 분야: 지방세특례제한법]
관련된 정보가 없습니다.

질문: 산업단지개발사업 및 산업기술단지의 사업시행자가 보유하는 부동산에 대해서 감면하는 경우도 있나요?


Querying with: [지방세 관련 질문 분야: 지방세특례제한법]
관련된 정보가 없습니다.

질문: 산업단지개발사업 및 산업기술단지의 사업시행자가 보유하는 부동산에 대해서 감면하는 경우도 있나요?

[ChatEngine 응답]

네, 지방세특례제한법에 따르면 산업단지개발사업이나 산업기술단지의 사업시행자가 취득하거나 보유하는 부동산에 대해 지방세를 감면하는 경우가 있습니다. 구체적으로는 산업단지나 산업기술단지를 조성하기 위해 취득하는 부동산에 대해서는 취득세와 재산세를 경감하며, 그 경감률은 수도권 외 지역에 있는 산업단지의 경우 더 높습니다. 또한, 산업용 건축물 등을 신축하기 위해 취득하는 토지와 신축 또는 증축하여 취득하는 산업용 건축물 등에 대해서도 취득세와 재산세를 경감하는데, 이 때 경감률은 동일합니다. 하지만 이런 감면 혜택을 받기 위해서는 일정 요건을 충족해야 하며, 그렇지 않은 경우 감면된 취득세 및 재산세를 추징당하게 됩니다.
